In [1]:
import pandas as pd
import numpy as np
import pickle
 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


In [2]:
df = pd.read_csv('merchant_data_project.csv')

In [3]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

for col in df.columns:
    if 'date' in col.lower():
        df[col] = pd.to_datetime(df[col], errors='coerce')

In [4]:
df.head()

,wallet_id,onboard_date,user_class,state,region,last_trans_date,first_trans_date,days_since_last_txn,days_to_first_txn,tenure_days,...,tpv_trend,avg_days_between_txn,pos_share,transfer_share,digital_service_share,loan_share,deposit_share,total_commission,success_rate,churn_flag
0,902ce7bd-40b4-48ff-9a4a-3b36ac11b9c8,2023-02-11,Terminal User,Lagos,Lagos,2024-05-27 11:28:00,2023-01-11 14:51:00,527,502,998,...,0.0,0.12,0.780,0.220,0.0018,0.0,0.0038,"20,933,029",0.86,1
1,1357e693-e92c-4614-aab0-6ad014b1c304,2024-03-21,Terminal User,Lagos,Lagos,2025-08-04 20:07:00,2024-03-27 09:31:00,93,495,594,...,0.0,0.19,0.033,0.085,0.7300,0.0,0.1500,"63,700",0.95,1
2,dfde92ee-fc91-4257-882c-ed1c44938d46,2023-02-11,App User,Lagos,Lagos,2023-02-11 23:32:00,2023-02-11 23:30:00,NaN,0,998,...,0.0,0.00,0.000,0.000,0.0000,0.0,0.0000,"5,685",0.00,1
3,852fb918-926f-4bc3-9c85-3a31d6f261db,2023-12-05,Terminal User,Kwara,South West,2025-01-04 17:55:00,2023-12-05 15:29:00,305,396,701,...,0.0,1.33,0.340,0.330,0.2500,0.0,0.0670,"403,650",0.82,1
4,63153e8f-3032-42eb-9d11-362be9ec88a0,2023-10-06,Terminal User,Lagos,Lagos,2024-10-07 17:58:00,2023-11-17 13:53:00,394,325,761,...,0.0,0.05,0.360,0.160,0.0000,0.0,0.4800,0,0.92,1


In [5]:
df.isna().sum()

wallet_id                   1
onboard_date                1
user_class                  0
state                     849
region                      0
last_trans_date          4835
first_trans_date         4835
days_since_last_txn      9995
days_to_first_txn        4835
tenure_days                 1
total_txn_count             0
total_tpv                   0
avg_tpv                     0
txn_count_30d               0
tpv_30d                     0
avg_txn_value_30d           0
txn_count_90d               0
tpv_90d                     0
avg_txn_value_90d           0
txn_trend                   0
tpv_trend                   0
avg_days_between_txn        0
pos_share                   0
transfer_share              0
digital_service_share       0
loan_share                  0
deposit_share               0
total_commission            0
success_rate                0
churn_flag                  0
dtype: int64

In [6]:
df.drop(columns=['onboard_date', 'first_trans_date', 'last_trans_date'], inplace=True)

In [7]:
df.dtypes

wallet_id                 object
user_class                object
state                     object
region                    object
days_since_last_txn       object
days_to_first_txn         object
tenure_days               object
total_txn_count           object
total_tpv                 object
avg_tpv                   object
txn_count_30d             object
tpv_30d                   object
avg_txn_value_30d         object
txn_count_90d             object
tpv_90d                   object
avg_txn_value_90d         object
txn_trend                float64
tpv_trend                float64
avg_days_between_txn     float64
pos_share                float64
transfer_share           float64
digital_service_share    float64
loan_share               float64
deposit_share            float64
total_commission          object
success_rate             float64
churn_flag                 int64
dtype: object

In [8]:
cols_to_convert = [
    'days_to_first_txn',
    'days_since_last_txn',
    'tenure_days',
    'total_txn_count',
    'total_tpv',
    'avg_tpv',
    'txn_count_30d',
    'tpv_30d',
    'avg_txn_value_30d',
    'txn_count_90d',
    'tpv_90d',
    'avg_txn_value_90d',
    'total_commission'
]

# Clean and convert all columns safely
for col in cols_to_convert:
    df[col] = (
        df[col]
        .astype(str)                        
        .str.replace(',', '', regex=False)  
        .str.replace('₦', '', regex=False)  
        .str.strip()                        
    )
    df[col] = pd.to_numeric(df[col], errors='coerce')  

df[cols_to_convert] = df[cols_to_convert].fillna(0)

In [9]:
df.dtypes

wallet_id                 object
user_class                object
state                     object
region                    object
days_since_last_txn      float64
days_to_first_txn        float64
tenure_days              float64
total_txn_count            int64
total_tpv                float64
avg_tpv                  float64
txn_count_30d              int64
tpv_30d                  float64
avg_txn_value_30d        float64
txn_count_90d              int64
tpv_90d                  float64
avg_txn_value_90d        float64
txn_trend                float64
tpv_trend                float64
avg_days_between_txn     float64
pos_share                float64
transfer_share           float64
digital_service_share    float64
loan_share               float64
deposit_share            float64
total_commission           int64
success_rate             float64
churn_flag                 int64
dtype: object

In [10]:
df.isna().sum()

wallet_id                  1
user_class                 0
state                    849
region                     0
days_since_last_txn        0
days_to_first_txn          0
tenure_days                0
total_txn_count            0
total_tpv                  0
avg_tpv                    0
txn_count_30d              0
tpv_30d                    0
avg_txn_value_30d          0
txn_count_90d              0
tpv_90d                    0
avg_txn_value_90d          0
txn_trend                  0
tpv_trend                  0
avg_days_between_txn       0
pos_share                  0
transfer_share             0
digital_service_share      0
loan_share                 0
deposit_share              0
total_commission           0
success_rate               0
churn_flag                 0
dtype: int64

In [11]:
df.drop(columns=['state', 'wallet_id'], inplace=True)

In [12]:
df.dtypes

user_class                object
region                    object
days_since_last_txn      float64
days_to_first_txn        float64
tenure_days              float64
total_txn_count            int64
total_tpv                float64
avg_tpv                  float64
txn_count_30d              int64
tpv_30d                  float64
avg_txn_value_30d        float64
txn_count_90d              int64
tpv_90d                  float64
avg_txn_value_90d        float64
txn_trend                float64
tpv_trend                float64
avg_days_between_txn     float64
pos_share                float64
transfer_share           float64
digital_service_share    float64
loan_share               float64
deposit_share            float64
total_commission           int64
success_rate             float64
churn_flag                 int64
dtype: object

In [13]:
df.isna().sum()

user_class               0
region                   0
days_since_last_txn      0
days_to_first_txn        0
tenure_days              0
total_txn_count          0
total_tpv                0
avg_tpv                  0
txn_count_30d            0
tpv_30d                  0
avg_txn_value_30d        0
txn_count_90d            0
tpv_90d                  0
avg_txn_value_90d        0
txn_trend                0
tpv_trend                0
avg_days_between_txn     0
pos_share                0
transfer_share           0
digital_service_share    0
loan_share               0
deposit_share            0
total_commission         0
success_rate             0
churn_flag               0
dtype: int64

In [14]:
df.isna().sum()

user_class               0
region                   0
days_since_last_txn      0
days_to_first_txn        0
tenure_days              0
total_txn_count          0
total_tpv                0
avg_tpv                  0
txn_count_30d            0
tpv_30d                  0
avg_txn_value_30d        0
txn_count_90d            0
tpv_90d                  0
avg_txn_value_90d        0
txn_trend                0
tpv_trend                0
avg_days_between_txn     0
pos_share                0
transfer_share           0
digital_service_share    0
loan_share               0
deposit_share            0
total_commission         0
success_rate             0
churn_flag               0
dtype: int64

In [15]:
df.head()

,user_class,region,days_since_last_txn,days_to_first_txn,tenure_days,total_txn_count,total_tpv,avg_tpv,txn_count_30d,tpv_30d,...,tpv_trend,avg_days_between_txn,pos_share,transfer_share,digital_service_share,loan_share,deposit_share,total_commission,success_rate,churn_flag
0,Terminal User,Lagos,527.0,502.0,998.0,3937,6.462156e+07,16413.91,0,0.0,...,0.0,0.12,0.780,0.220,0.0018,0.0,0.0038,20933029,0.86,1
1,Terminal User,Lagos,93.0,495.0,594.0,2548,2.168474e+07,8510.49,0,0.0,...,0.0,0.19,0.033,0.085,0.7300,0.0,0.1500,63700,0.95,1
2,App User,Lagos,0.0,0.0,998.0,0,0.000000e+00,0.00,0,0.0,...,0.0,0.00,0.000,0.000,0.0000,0.0,0.0000,5685,0.00,1
3,Terminal User,South West,305.0,396.0,701.0,299,1.670792e+06,5587.93,0,0.0,...,0.0,1.33,0.340,0.330,0.2500,0.0,0.0670,403650,0.82,1
4,Terminal User,Lagos,394.0,325.0,761.0,6307,2.399791e+08,38049.64,0,0.0,...,0.0,0.05,0.360,0.160,0.0000,0.0,0.4800,0,0.92,1


In [16]:
df['churn_flag'].unique()

array([1, 0])

In [17]:
df

,user_class,region,days_since_last_txn,days_to_first_txn,tenure_days,total_txn_count,total_tpv,avg_tpv,txn_count_30d,tpv_30d,...,tpv_trend,avg_days_between_txn,pos_share,transfer_share,digital_service_share,loan_share,deposit_share,total_commission,success_rate,churn_flag
0,Terminal User,Lagos,527.0,502.0,998.0,3937,6.462156e+07,16413.91,0,0.0,...,0.0,0.12,0.780,0.220,0.0018,0.0,0.0038,20933029,0.86,1
1,Terminal User,Lagos,93.0,495.0,594.0,2548,2.168474e+07,8510.49,0,0.0,...,0.0,0.19,0.033,0.085,0.7300,0.0,0.1500,63700,0.95,1
2,App User,Lagos,0.0,0.0,998.0,0,0.000000e+00,0.00,0,0.0,...,0.0,0.00,0.000,0.000,0.0000,0.0,0.0000,5685,0.00,1
3,Terminal User,South West,305.0,396.0,701.0,299,1.670792e+06,5587.93,0,0.0,...,0.0,1.33,0.340,0.330,0.2500,0.0,0.0670,403650,0.82,1
4,Terminal User,Lagos,394.0,325.0,761.0,6307,2.399791e+08,38049.64,0,0.0,...,0.0,0.05,0.360,0.160,0.0000,0.0,0.4800,0,0.92,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23591,App User,South West,1105.0,0.0,1107.0,1,7.960000e+03,7960.00,0,0.0,...,0.0,0.00,1.000,0.000,0.0000,0.0,0.0000,0,0.00,1
23592,App User,South West,0.0,0.0,685.0,0,0.000000e+00,0.00,0,0.0,...,0.0,0.00,0.000,0.000,0.0000,0.0,0.0000,0,0.00,0
23593,App User,South East,180.0,90.0,276.0,29,9.110000e+03,314.14,0,0.0,...,0.0,3.21,0.000,0.550,0.2800,0.0,0.1700,110200,0.72,1
23594,App User,South West,937.0,91.0,998.0,4,2.600000e+04,6500.00,0,0.0,...,0.0,18.33,0.000,0.500,0.2500,0.0,0.2500,3360,1.00,1


In [18]:
df.describe()

,days_since_last_txn,days_to_first_txn,tenure_days,total_txn_count,total_tpv,avg_tpv,txn_count_30d,tpv_30d,avg_txn_value_30d,txn_count_90d,...,tpv_trend,avg_days_between_txn,pos_share,transfer_share,digital_service_share,loan_share,deposit_share,total_commission,success_rate,churn_flag
count,23596.000000,23596.000000,23596.000000,23596.000000,2.359600e+04,2.359600e+04,23596.000000,2.359600e+04,23596.000000,23596.000000,...,23596.000000,23596.000000,23596.000000,23596.000000,23596.000000,23596.000000,23596.000000,2.359600e+04,23596.000000,23596.000000
mean,284.757417,221.030683,861.812256,683.956645,2.158054e+07,2.291652e+04,5.413375,7.900463e+04,950.992812,15.967706,...,0.030400,2.113905,0.122460,0.141749,0.203005,0.000283,0.104914,2.155934e+08,0.513607,0.707154
std,359.956360,320.625447,249.321576,2004.799540,1.304035e+09,1.712576e+06,37.357900,9.875321e+05,6080.427022,113.014642,...,0.127292,13.922280,0.221773,0.204980,0.293044,0.006263,0.176583,7.992625e+09,0.448105,0.455078
min,0.000000,0.000000,0.000000,0.000000,-2.000000e+01,-2.000000e+01,0.000000,-4.000000e+00,-4.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.120000e+06,0.000000,0.000000
25%,0.000000,0.000000,750.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
50%,31.000000,24.000000,998.000000,5.000000,1.000000e+04,1.100000e+03,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.100000,0.000000,0.003900,0.000000,0.000000,0.002000,5.250000e+03,0.800000,1.000000
75%,586.000000,367.000000,998.000000,366.000000,3.926660e+06,9.726943e+03,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.840000,0.170000,0.250000,0.370000,0.000000,0.160000,7.282200e+05,0.920000,1.000000
max,1346.000000,1289.000000,1353.000000,59057.000000,2.000157e+11,2.624878e+08,1462.000000,8.858314e+07,183239.900000,5045.000000,...,1.000000,894.000000,1.000000,1.000000,1.000000,0.330000,1.000000,8.592304e+11,1.000000,1.000000


In [19]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [20]:
categorical = list(df.select_dtypes(include=['object']).columns)
numerical = list(df.select_dtypes(exclude=['object']).columns)

#numerical = numerical.drop('churn_flag', errors='ignore')
numerical = [col for col in numerical if col != 'churn_flag']

In [21]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')
 
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)
 
    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
 
    return dv, model

In [22]:
def predict(df, dv, model):
     dicts = df[categorical + numerical].to_dict(orient='records')
 
     X = dv.transform(dicts)
     y_pred = model.predict_proba(X)[:,1]
 
     return y_pred

In [23]:
C = 1.0
n_splits = 5

In [24]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)  
 
scores = []
 
for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]
 
    y_train = df_train.churn_flag.values
    y_val = df_val.churn_flag.values
 
    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)
 
    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)
 
print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessi

C=1.0 0.965 +- 0.003


/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [25]:
scores

[0.9659797133311205,
 0.967238602920762,
 0.9646871469299315,
 0.9605376997991608,
 0.967396520572131]

In [26]:
dv, model = train(df_full_train, df_full_train.churn_flag.values, C=1.0)
y_pred = predict(df_test, dv, model)
y_test = df_test.churn_flag.values
 
auc = roc_auc_score(y_test, y_pred)
auc

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9645537700473498

In [27]:
auc

0.9645537700473498

In [28]:
output_file = f'model_C={C}.bin'
output_file

'model_C=1.0.bin'

In [29]:
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

In [31]:
df.iloc[23592].count()

np.int64(25)

In [35]:
df.iloc[23]

user_class               Terminal User
region                     South South
days_since_last_txn               90.0
days_to_first_txn                700.0
tenure_days                      790.0
total_txn_count                   2381
total_tpv                  17899332.25
avg_tpv                        7517.57
txn_count_30d                        0
tpv_30d                            0.0
avg_txn_value_30d                  0.0
txn_count_90d                        1
tpv_90d                          400.0
avg_txn_value_90d                400.0
txn_trend                          0.0
tpv_trend                          0.0
avg_days_between_txn              0.29
pos_share                         0.45
transfer_share                    0.18
digital_service_share             0.34
loan_share                         0.0
deposit_share                     0.03
total_commission             541867980
success_rate                      0.87
churn_flag                           1
Name: 23, dtype: object